In [1]:
# Importação das bibliotecas necessárias
# - pandas (para manipulação de dados)
# - matplotlib e seaborn (para visualização de dados)
# - sklearn.preprocessing (para pré-processamento dos dados)
# - sklearn.cluster (para o algoritmo de clusterização K-Means)
# - sklearn.neighbors (para a classificação usando KNN)
# - sklearn.model_selection (para separar os dados de treino e teste)
# - sklearn.metrics (para calcular métricas de avaliação, como o score de Silhouette)
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

In [2]:
# Leitura do arquivo CSV que contém os dados do cliente para análise de crédito
df = pd.read_csv('solicitacoescredito.csv')
# A linha comentada poderia salvar os dados em formato Excel, se necessário.
# df.to_excel('solicitacoescredito.xlsx')

## Filtro das colunas para melhorar a precisão do nosso modelo

In [3]:
# Seleção das colunas mais importantes para a análise de crédito
# Essas colunas podem conter informações sobre o risco de crédito e comportamento financeiro
colunas_importantes = [
    "maiorAtraso",  # Tempo de maior atraso nos pagamentos
    "margemBrutaAcumulada",  # Margem bruta acumulada da empresa
    "percentualProtestos",  # Percentual de protestos financeiros contra a empresa
    "prazoMedioRecebimentoVendas",  # Prazo médio de recebimento das vendas
    "titulosEmAberto",  # Títulos financeiros ainda em aberto
    "valorSolicitado",  # Valor de crédito solicitado pela empresa
    "status",  # Status atual da solicitação de crédito
    "definicaoRisco",  # Definição do risco atribuída à empresa
    "percentualRisco",  # Percentual de risco associado à empresa
    "valorAprovado",  # Valor de crédito aprovado
    "endividamento",  # Nível de endividamento da empresa
    "faturamentoBruto",  # Faturamento bruto da empresa
    "limiteEmpresaAnaliseCredito",  # Limite de crédito atribuído pela análise
    "restricoes",  # Restrições financeiras da empresa
    "scorePontualidade"  # Pontuação de pontualidade de pagamento
]

In [ ]:
# Filtragem do DataFrame para manter apenas as colunas importantes e remoção de dados ausentes
df_filtrado = df[colunas_importantes]
df_filtrado = df_filtrado.dropna()

In [4]:
# Codificação de variáveis categóricas para numéricas usando LabelEncoder
# O 'status' e 'definicaoRisco' são convertidos para números
le_status = LabelEncoder()
le_definicaoRisco = LabelEncoder()

df_filtrado['status'] = le_status.fit_transform(df_filtrado['status'])
df_filtrado['definicaoRisco'] = le_definicaoRisco.fit_transform(df_filtrado['definicaoRisco'])

In [6]:
# Pré-processamento dos dados: Normalização
# Os dados são normalizados para que todas as variáveis tenham a mesma escala,
# evitando que variáveis com maiores magnitudes dominem o modelo
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_filtrado.dropna())

In [7]:
# Clusterização com K-Means
# Aplicação do algoritmo de K-Means para clusterização
# O algoritmo agrupa os dados em 5 clusters diferentes

kmeans = KMeans(n_clusters=5, random_state=42)
df_filtrado['cluster'] = kmeans.fit_predict(X_scaled)


In [8]:
# Avaliação da qualidade dos clusters usando a métrica Silhouette
# O score de Silhouette mede a coesão e separação dos clusters
silhouette_avg = silhouette_score(X_scaled, df_filtrado['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.2507798699897532


In [9]:
# Divisão dos dados em treino e teste para classificação usando KNN
# Aqui, 80% dos dados são usados para treino e 20% para teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df_filtrado['cluster'], test_size=0.2, random_state=42)

KNeighborsClassifier()

In [ ]:
# Treinamento do modelo KNN com 5 vizinhos para classificar novos clientes
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [10]:
# Avaliação do modelo KNN com a acurácia nos dados de teste
accuracy = knn.score(X_test, y_test)
print(f"Acurácia do KNN: {accuracy}")

Acurácia do KNN: 0.9850136239782016


In [11]:
# Cálculo da média do limite de crédito por cluster, baseado nos clusters formados
# Isso pode ser usado para recomendar limites de crédito para novos clientes, dependendo do cluster em que eles caem
cluster_limits = df_filtrado.groupby('cluster')['limiteEmpresaAnaliseCredito'].mean()
print(cluster_limits)

cluster
0    6.324979e+05
1    5.744829e+06
2    3.418744e+06
3    1.347838e+06
4    9.095663e+07
Name: limiteEmpresaAnaliseCredito, dtype: float64
